# t265_demo

Notebook using a 2D plot representing the t265 positioning with buttons to move the robot.

In [ ]:
# Insérez ici le numéro de votre équipe
# (en) Insert here your team number
VOTRE_NUMERO_EQUIPE = 0

In [ ]:
# Import des paquets requis et configuration du ROS Master
# (en) Import of required packages and configuration of the ROS Master
import sys; sys.path.append('/opt/ros/melodic/lib/python2.7/dist-packages/')
from os import environ
environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
import rospy
from jackal_msgs.msg import Drive
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Pose
import numpy as np

In [ ]:
# Créer et démarrer un nouveau noeud
# (en) Create and start a new node
rospy.init_node('dingo_controller', anonymous=True)

## Création des 'callbacks' (Create callbacks)

Chaque fonction est associée à un sujet ROS et enregistre en continue sont contenu dans une variable globale.

(en) Each function is associated with a ROS topic and continuously records its content in a global variable.

In [ ]:
# Realsense Pose subscriber callback
pose_msg = Pose()
def pose_callback(msg):
    global pose_msg
    pose_msg  = msg.pose.pose
# Fonction pour le calcul de l'orientation à partir d'un quaternion
# (en) Function for calculating the orientation from a quaternion
from scipy.spatial.transform import Rotation as R
def get_heading_from_quaternion(q):
    r = R.from_quat([q.x, q.y, q.z, q.w])
    angles = r.as_euler('xyz', degrees=False)
    return angles[2]
def wraptopi(angle):
    xwrap=np.remainder(angle, 2*np.pi)
    if np.abs(xwrap)>np.pi:
        xwrap -= 2*np.pi * np.sign(xwrap)
    return xwrap

In [ ]:
# ROS subscribers et publishers
cmd_drive_pub = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
pose_sub = rospy.Subscriber('/mobile_manip/t265/odom/sample', Odometry, pose_callback)

## Contrôleur du mouvement (Movement controller)

In [ ]:
interwheel_distance = 0.3765
left_wheel_radius = 0.098
right_wheel_radius = 0.098
def move_robot(linear, angular):
    vel_left  = (linear - angular * interwheel_distance / 2.0) / left_wheel_radius
    vel_right = (linear + angular * interwheel_distance / 2.0) / right_wheel_radius

    # Envoi des commandes au roues par topic ROS
    # (en) Sending commands to the wheels via ROS topic
    cmd_drive_msg = Drive()
    cmd_drive_msg.drivers[0] = vel_left
    cmd_drive_msg.drivers[1] = vel_right
    cmd_drive_pub.publish(cmd_drive_msg)

## Interface manuelle (Manual interface)

Permet le contrôle du robot avec des flèches et un retour visuel de la pose obtenue par la caméra T265.

(en) Allows the control of the robot with arrows and a visual feedback of the pose obtained by the T265 camera.

In [ ]:
%matplotlib notebook
import ipywidgets as widgets
import threading
import time, math
from matplotlib import pyplot as plt

btn_up = widgets.Button(icon='arrow-up')
btn_left = widgets.Button(icon='arrow-left')
btn_down = widgets.Button(icon='arrow-down')
btn_right = widgets.Button(icon='arrow-right')
btn_stop = widgets.Button(description='Arret')
btn_eff = widgets.Button(description='Effacer')

fig = plt.figure(figsize=(6,3))
XY = []

def update_plot():
    global fig, XY
    while True:
        # Calcul du vecteur
        cap = wraptopi(get_heading_from_quaternion(pose_msg.orientation))
        x_pos = pose_msg.position.x
        y_pos = pose_msg.position.y
        dx = math.cos(cap)
        dy= math.sin(cap)
        XY.append([x_pos, y_pos])
        # Génération de la visualtion
        plt.clf()
        plt.scatter(np.asarray(XY)[:,0],np.asarray(XY)[:,1], color='y')
        plt.arrow(x_pos, y_pos, dx, dy, color="#aa0088")
        plt.axis([-10, 10, -10, 10])
        plt.grid(True)
        fig.canvas.draw_idle()
        plt.pause(0.1)

def on_btn_up_clicked(b):
    move_robot(2,0)

def on_btn_left_clicked(b):
    move_robot(0,1)

def on_btn_down_clicked(b):
    move_robot(-2,0)

def on_btn_right_clicked(b):
    move_robot(0,-1)

def on_btn_stop_clicked(b):
    move_robot(0,0)

def on_btn_eff_clicked(b):
    global XY
    XY = []

btn_up.on_click(on_btn_up_clicked)
btn_left.on_click(on_btn_left_clicked)
btn_down.on_click(on_btn_down_clicked)
btn_right.on_click(on_btn_right_clicked)
btn_stop.on_click(on_btn_stop_clicked)
btn_eff.on_click(on_btn_eff_clicked)


grid = widgets.GridspecLayout(4, 3,height='500px')

grid[0,0] = btn_eff
grid[1,1] = btn_up
grid[2,0] = btn_left
grid[2,1] = btn_stop
grid[2,2] = btn_right
grid[3,1] = btn_down
thread = threading.Thread(target=update_plot)
thread.start()
grid